## ***

Перебирая различную глубину дерева на один и тех же тестовых данных, мы не решаем проблему переобучения. Исправить ситуацию поможет  кросс-валидация (перекрёстная проверка), разбивающая данные на тренировочную выборку для обучения, валидирующее множество для подбора оптимальных параметров и тестовые данные для финального замера точности.

Что мы делаем:

1. Разделяем данные (X_train, y_train) на 5 частей, чтобы одну часть (всегда разную) использовать как тестовые данные, а остальные 4 как тренировочные. Получится, что каждый кусок данных будет использован и в обучении, и в тесте. Для каждой модели усредним accuracy на подвыборках, чтобы потом использовать как единый показатель точности для сравнения разных моделей.

```
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)
from sklearn.model_selection import cross_val_score
cross_val_score(clf, X_train, y_train , cv=5) # array([0.74166667, 0.8     , 0.75630252, 0.79831933, 0.74576271])
cross_val_score(clf, X_train, y_train , cv=5).mean() # 0.7734524996439254
```

2. Возвращаемся к работе с циклом для поиска оптимальной глубины и добавляем в scores_data среднюю точность на кросс-валидации. Чтобы не было путаницы в одном ноутбуке переименуем scores_data в cross_val_scores_data.

```
cross_val_scores_data = pd.DataFrame()
for max_depth in max_depth_values:
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test)
   
    mean_cross_val_score = cross_val_score(clf, X_train, y_train , cv=5).mean()
   
    temp_score_data = pd.DataFrame({'max_depth': [max_depth],
                                    'train_score': [train_score],
                                    'test_score': [test_score],
                                    'cross_val_score': [mean_cross_val_score]})
    cross_val_scores_data = cross_val_scores_data.append(temp_score_data)
```

3. В результате получим датафрейм с колонкой max_depth и тремя score, используя функцию melt, преобразуем данные и построим график:

```
cross_val_scores_data_long = pd.melt(cross_val_scores_data, id_vars=['max_depth'], value_vars=['train_score', 'test_score', cross_val_score'], var_name='set_type', value_name='score')
sns.lineplot(x="max_depth", y="score", hue="set_type", data=cross_val_scores_data_long)
```

График может немного отличаться, потому что  cross_val_score случайно разбивает данные на 5 частей.

4. Смотрим на зеленую линию (cross_val_score ). На графике ясно видно, что максимум приходится на глубину до 20. более точное значение максимальной глубины дерева поможет определить.

```
cross_val_scores_data_long.query("set_type == 'cross_val_score'").head(20)
```

График может немного отличаться.

5. Создаем классификатор, указывая соответствующее значение аргумента  max_depth. Обучаем на тренировочной выборке и замеряем точность на тестовой.

```
best_clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10)
best_clf.fit(X_train, y_train)
best_clf.score(X_test, y_test)
```


## ***

Перекрёстная проверка не возвращает модель: кросс-валидация не является способом построения модели, которую можно применить к новым данным. При вызове ```cross_val_score``` строится несколько внутренних моделей, однако цель перекрёстной проверки заключается только в том, чтобы оценить обобщающую способность данного алгоритма, обучив на определённом наборе данных. ("Введение в машинное обучение с помощью Python. Руководство для специалистов по работе с данными" А.Мюллер, С.Гвидо)

При k-fold кросс-валидации обучающая выборка разбивается на (k-1) подвыборок и следующий алгоритм выполняется k раз:
* Модель обучается с использованием (k-1) подвыборок в качестве тренировочных данных;
* Полученная модель проверяется на оставшейся одной подвыборке (которая служит тестовым набором для вычисления точности). 

Модель – это набор подготовленных данных (переменных) + метод (статистики, машинного обучения), подобранные для достижения цели исследования (проекта Data Science)

Процесс моделирования включает:
1) Создание модели – планирование показателей (переменных) и выбор метода
2) Тренировка модели
3) Проверка адекватности и окончательный выбор модели
4) Применение тренированной модели к незнакомым данным

Хорошая модель будет найдена (вероятно) только после многократного повторения первых 3-х шагов. ("Основы Data Science и Big Data. Python и наука о данных" Д.Силен, А.Мейсман, М.Али)

Таким образом,
1. Нужна хорошая модель, которая позволит достичь целей проекта, которую нужно обучить, протестировать, оценить и окончательно выбрать, после чего её можно будет применять "на производстве"
2. В данном уроке модель – это предобработанные данные Titanic + метод моделирования "Дерево решений", модель обучается методом fit() и тестируется 2-мя способами: score и cross_val_score
3. Метод cross_val_score() содержит (условно) процедуры обучения (fit) и проверки (score) , но он не может возвращать модель, которую можно применять к новым данным.

## ***

1) Точность (score) как процент правильных ответов не хорошая и не верная метрика качества модели. Например, в ситуации, когда только 10% выживших, любая модель-классификатор , которая бы всем предсказывала, что они не выживут, давала бы точность 90% , так как 10% выживших здесь, по сути, погрешность, равная доле неправильно классифицированных.

2) Точность хорошо работает на сбалансированных выборках, т.е. когда равномерное распределение между классами.

3) Получается, точность как доля верно классифицированных может быть разной для разных классов.

## ***

Работа с несбалансированными выборками

Можно считать, что выборка несбалансирована, когда размеры классов отличаются более, чем в 10 раз.

Больший класс называют доминирующим, меньший класс называется минорным.

Зачастую можно повысить качество с помощью
* корректировки весов объектов;
* исскуственной модификации датасета.

## ***

Типы ошибок:
* type I  – false positive – you're pregnant
* type II – false negative – you're not pregnant

## Метрики качества модели

Пример с космолётами хорошо показывает 2 ситуации, когда наш классификатор может ошибаться:

С одной стороны мы можем отлично решать задачу нахождения вражеских космолётов, но платить за это слишком высокую цену. С другой стороны мы сильно избежим ложных срабатываний, но при этом не отловим все положительные кейсы.

Precision и Recall - это метрики, отвечающие на вопрос как хорошо нашему классификатору удаётся справляться с двумя сторонами задачи классификации.

Precision – точность.
Она отвечает на вопрос "насколько хорошо у нас получается находить положительные классы, не переплачивая за это ложными срабатываниями?".

Данная метрика не отвечает на вопрос насколько хорошо в целом у нас получается находить положительные примеры.

Recall – полнота.
Она нужна, поскольку нам интересно не только то как хорошо мы можем находить положительные примеры без ложных срабатываний, но и как хорошо в целом мы находим положительные примеры.

Она отвечает на вопрос "правда ли, что мы нашли всех представителей положительного класса?".

## Вопрос: так а что же более важное?

Пусть у нас есть всего 2 космолёта, и мы не можем себе позволить отправить их на ложное срабатывание. Тогда нам более важна метрика Precision. Recall нам не важен.

Обратная ситуация: мы играем за империюи у нас безграничные ресурсы, а у противника всего 2 корабля. Тогда нам не важно ложное срабатывание, нам нужо любой ценой найти 2 корабля вражеского сопротивления, т.е. мы оптимизируем Recall.

## ***

Алгоритм - это пастух, лайки Анатолия - волки.

1. Пастух сказал что волки есть в 30 случаях из 100.
2. На самом деле волки были в 15 случаях из этих 30 (те самые лайки Анатолия). Это как раз и есть "верное срабатывание" или True Positive.
3. Соответственно ситуация, когда пастух сказал что волки есть, а их не было называется False Positive в данном случае это тоже 15, т.к. 15 раз мы сказали что волки есть (фото понравится), но ошиблись - волков нет (фото НЕ понравилось). Иными словами "ложное срабатывание".
4. Из 70 оставшихся случаев (фото) - 30 раз волки были (лайк на фото), но пастух сказал что волков нет (лайков не будет) - это False Negative, то есть "пропуск события".
5. Далее осталось 40 случаев (фото), когда пастух сказал что волков нет (лайков не будет) и их действительно не было. Это True Negative получается что-то типа "верное бездействие".

## ***

* precision более важен в ситуациях, где не нужны ложные положительные срабатывания, 
* а recall - там, где не нужны ложные отрицательные.

Например, если руководство страны пытается предотвратить эпидемию, и решило бесплатно выдавать лекарства всем заболевшим, то более важно будет покрыть всю заболевшую аудиторию == минимизировать случаи, когда больной считается здоровым == увеличить recall

* precision - главное не ошибиться
* recall - главное не пропустить

## ROC. Receiver Operating Characteristic

* ROC-кривая позволяет отобразить на одном графике результаты большого числа матриц ошибок (confusion matrix) в зависимости от различного уровня порога отсечения бинарных классов (хотя дерево решений может использоваться и для небинарной классификации).


* Ось y – TPR – Recall = TP/(TP+FN).
* На ось X может быть выведен 
** или FPR = 1-Specificity = 1-TN/(TN+FP), 
** или Precision=TP/(TP+FP)
** тот или иной показатель используется в зависимости от отсутствия или наличия дисбаланса классов.


* AUC = area under the curve = интегральный показатель площади под кривой [0;1], 
* AUC=0.5 - дерево не лучше случайного распределения

Чтобы нарисовать ROC-кривую, надо взять единичный квадрат на координатной плоскости, см. рис. 1, разбить его на m равных частей горизонтальными линиями и на n – вертикальными, где m – число 1 среди правильных меток теста (в нашем примере m=3), n – число нулей (n=4). В результате квадрат разбивается сеткой на m×n блоков.

## AUC ROC

https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/

Часто результат работы алгоритма на фиксированной тестовой выборке визуализируют с помощью ROC-кривой (ROC = receiver operating characteristic, иногда говорят «кривая ошибок»), а качество оценивают как площадь под этой кривой – AUC (AUC = area under the curve). Покажем на конкретном примере, как строится кривая.

Пусть алгоритм выдал оценки, как показано в табл. 1. Упорядочим строки табл. 1 по убыванию ответов алгоритма – получим табл. 2. Ясно, что в идеале её столбец «класс» тоже станет упорядочен (сначала идут 1, потом 0); в самом худшем случае – порядок будет обратный (сначала 0, потом 1); в случае «слепого угадывания» будет случайное распределение 0 и 1.

![](https://alexanderdyakonov.files.wordpress.com/2017/07/table.png)

Чтобы нарисовать ROC-кривую, надо взять единичный квадрат на координатной плоскости, см. рис. 1, разбить его на m равных частей горизонтальными линиями и на n – вертикальными, где m – число 1 среди правильных меток теста (в нашем примере m=3), n – число нулей (n=4). В результате квадрат разбивается сеткой на m×n блоков.

Теперь будем просматривать строки табл. 2 сверху вниз и прорисовывать на сетке линии, переходя их одного узла в другой. Стартуем из точки (0, 0). Если значение метки класса в просматриваемой строке 1, то делаем шаг вверх; если 0, то делаем шаг вправо. Ясно, что в итоге мы попадём в точку (1, 1), т.к. сделаем в сумме m шагов вверх и n шагов вправо.

![](https://alexanderdyakonov.files.wordpress.com/2017/07/pic1.png?w=399&h=189)

На рис. 1 (справа) показан путь для нашего примера – это и является ROC-кривой. Важный момент: если у нескольких объектов значения оценок равны, то мы делаем шаг в точку, которая на a блоков выше и b блоков правее, где a – число единиц в группе объектов с одним значением метки, b – число нулей в ней. В частности, если все объекты имеют одинаковую метку, то мы сразу шагаем из точки (0, 0) в точку (1, 1).

Выше мы описали случаи идеального, наихудшего и случайного следования меток в упорядоченной таблице. Идеальному соответствует ROC-кривая, проходящая через точку (0, 1), площадь под ней равна 1. Наихудшему – ROC-кривая, проходящая через точку (1, 0), площадь под ней – 0. Случайному – что-то похожее на диагональ квадрата, площадь примерно равна 0.5.

![](https://alexanderdyakonov.files.wordpress.com/2017/07/pic2.png?w=433&h=142)

Замечание. ROC-кривая считается неопределённой для тестовой выборки целиком состоящей из объектов только одного класса. Большинство современных реализаций выдают ошибку при попытки построить её в этом случае

## Смысл AUC ROC

Сетка на рис. 1 разбила квадрат на m×n блоков. Ровно столько же пар вида (объект класса 1, объект класса 0), составленных из объектов тестовой выборки. Каждый закрашенный блок на рис. 1 соответствует паре (объект класса 1, объект класса 0), для которой наш алгоритм правильно предсказал порядок (объект класса 1 получил оценку выше, чем объект класса 0), незакрашенный блок – паре, на которой ошибся.

![](https://alexanderdyakonov.files.wordpress.com/2017/07/pic3.png?w=197&h=181)

Таким образом, AUC ROC равен доле пар объектов вида (объект класса 1, объект класса 0), которые алгоритм верно упорядочил, т.е. первый объект идёт в упорядоченном списке раньше. Численно это можно записать так:

![](https://alexanderdyakonov.files.wordpress.com/2017/07/eq.png)

Замечание. В формуле (*) все постоянно ошибаются, забывая случай равенства ответов алгоритма на нескольких объектах. Также эту формулу все постоянно переоткрывают. Она хороша тем, что легко обобщается и на другие задачи обучения с учителем.

## Принятие решений на основе ROC-кривой

Пока наш алгоритм выдавал оценки принадлежности к классу 1. Ясно, что на практике нам часто надо будет решить: какие объекты отнести к классу 1, а какие к классу 0. Для этого нужно будет выбрать некоторый порог (объекты с оценками выше порога считаем принадлежащими классу 1, остальные – 0).

## ***

Дерево решений - алгоритм машинного обучения для регрессии и классификации. Остановимся подробнее на задаче классификатора. В процессе построения дерева выбираются переменные разделение, по которым позволяет нам с большей точностью разделять исследуемые классы. В каждом листе дерева находится одно условие, которое разбивает наш набор данных на два подмножества.

Процесс выбора фичей, которые помещаются в лист дерева основывается на расчете Information gain. Могут использоваться и другие критерии.

По умолчанию классификатор строит настолько глубокое дерево, насколько позволяет набор данных. Таким образом, классификатор преследует цель безошибочно отнести к нужному классу все объекты в выборке. Такой "жадный" алгоритм может приводить к переобучению в результате избыточной глубины дерева.

Проверить насколько сильно переобучен классификатор можно обучив его на обучающей выборке затем проверить качество модели на тестовой выборке.
Основная цель построения любой модели - получить классификатор с максимальной обобщающей способностью, т.е. такой классификатор, который успешно выделяет общие закономерности и не переобучается на частных закономерностях в конкретной выборке.

Для этой цели мы подбираем оптимальный набор параметров классификатора, в случае дерева решений - это может быть глубина дерева. Для выбора оптимального параметра можно построить множество деревьев с разной глубиной и выбрать наилучший результат.

В качестве критерия "наилучшего" классификатора можно использовать кросс-валидацию. При таком подходе данные в выборке разбиваются на n частей, например на 5 частей, далее классификатор обучается на 4 частях данных и валидируется на оставшейся 5ой части. Затем операция повторяется, чтобы каждая из 5 частей данных оказалась в роли тестового множества.

## min_samples_split

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

С помощью этого параметра мы задаём, какое минимальное количество элементов должно остаться в очередной ветке дерева, чтобы алгоритм считал, что эту ветку всё ещё можно продолжать разделять. 

Стандартное значение этого параметра равно двум, что естественно: пока в ветке есть хотя бы два элемента разных классов, мы можем продолжать искать способ разделить их. Однако, мы можем увеличить этот параметр, например, до трёх. Тогда ветка с двумя элементами уже будет считаться неразделимой. Тут мы теряем в точности на тренировочной выборке, потому что возникает вероятность того, что в одной такой ветке останутся элементы разных классов, но зато мы выигрываем в генерализации (возможно) и интерпретируемости, так как рост дерева вглубину ограничивается и оно становится проще.

min_samples_split - ограничитель, который не дает формироваться слишком "тонким" узлам у дерева

min_samples_leaf - ограничитель, который не дает формироваться слишком "маленьким" листикам у дерева

Мера величины узла или листа - это количество "сэмплов" элементов выборки, находящиеся в узле или листе.

min_samples_split = 5, min_samples_leaf = 3

Растет дерево (обучается модель). Отпочковывается лист с 10-ю элементами. Лист удовлетворяет ограничениям, алгоритм после анализа определяет в нем 4 отрицательных, 6 положительных элементов. Лист превращается в узел и дает 2 листа: 4 и 6 элементов. 4-элементный лист более не анализируется на предмет разделения так как попадает под ограничение min_samples_split. Над 6-эл. проводится анализ, в результате которого алгоритм предлагает разбиение - 2 отрицательных и 4 положительных. Разбиения не происходит, так как срабатывает ограничение min_samples_leaf (лист не может содержать менее 3 элементов).

   ...
     \
   10 (4,6)
  /        \
4 (...)   6 (2,4)

# Стратегии разделения датасета для тестирования

### Leave P Out (LPO)

* LeavePOut is very similar to LeaveOneOut as it creates all the possible training/test sets by removing p samples from the complete set. For n samples, this produces (np) train-test pairs. Unlike LeaveOneOut and KFold, the test sets will overlap for p>1.


* Разделение данных на 2 части с n−k и k наблюдениями, на первой идёт тренировка, 2-ая - для предсказания; идёт ротация по всем наблюдениям


* У вас 100 наблюдений (это n). Вы можете определить, что хотите проверять на произвольном числе наблюдений. Например, на 20 (это k). Тренируете соответственно на 80 (это n-k). И так проходится по всем наблюдениям, набирая разные составы 80-2

### Leave One Out (LOO)

* LeaveOneOut (or LOO) is a simple cross-validation. Each learning set is created by taking all the samples except one, the test set being the sample left out. Thus, for n samples, we have n different training sets and n different tests set. This cross-validation procedure does not waste much data as only one sample is removed from the training set:


* Разделение данных на 2 части с n−1 и 1 наблюдением, на первой идёт тренировка, 2-ая - для предсказания; каждое наблюдение побывает во второй части


* У вас 100 наблюдений. Тренируете на 99, проверяете на 1. В качестве проверочного по очереди побывают все наблюдения.

### cross_validate

* Разделение данных на k частей, тренировка на k−1 частях, тестирование на оставшейся; так для каждой части


* У вас 100 наблюдений. Вы делите их на произвольное количество частей. Например, на 4. Каждая по 25 наблюдений. Тренируете на 3 частях (75 наблюдений), проверяете на 1 части (25 наблюдений). И по очереди тасуются все части, попадая в проверочную часть.

### ShuffleSplit

* Аналог обычного разделения на тестовый и тренировочный датасеты с большим числом таких случайных разделений

### train_test_split

* Разделение имеющихся данных на тестовый и тренировочный наборы

# Описание метрик

### Specificity, TrueNegativeRate
отношение правильно определённых отрицательных случаев ко всем отрицательным – показывает какую часть отрицательных случаев модель правильно классифицирует

### Recall, TruePositiveRate, Sensitivity
отношение правильно определённых положительных случаев ко всем положительным – показывает какую часть положительных случаев модель правильно классифицирует

### F1 − score
отношение удвоенного произведения precision и recall к их сумме

### Precision
отношение правильно определённых положительных наблюдений ко всем определённым как положительные

### Accuracy
отношение правильно классифицированных наблюдений ко всем

# Параметры функции DecisionTreeClassifier

#### min_impurity_decrease
Минимальное снижение "нечистоты" (смешения классов) узла при разделении, чтобы разделение произошло

#### min_samples_leaf
Минимальное число образцов в листьях (при получившемся значении ниже разделение не будет произведено)

#### max_depth
Максимальное число уровней дерева (максимальная длина пути от корня до листа)

#### min_samples_split
Минимальное число образцов в узле, чтобы его можно было разделить на 2